
<br>
===============================<br>
Using geometric transformations<br>
===============================<br>
In this example, we will see how to use geometric transformations in the context<br>
of image processing.<br>


In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from skimage import data
from skimage import transform

####################################################################<br>
Basics<br>
======<br>
<br>
Several different geometric transformation types are supported: similarity,<br>
affine, projective and polynomial. For a tutorial on the available types of<br>
transformations, see :ref:`sphx_glr_auto_examples_transform_plot_transform_types.py`.<br>
<br>
Geometric transformations can either be created using the explicit<br>
parameters (e.g. scale, shear, rotation and translation) or the<br>
transformation matrix.<br>
<br>
First we create a transformation using explicit parameters:

In [ ]:
tform = transform.SimilarityTransform(scale=1, rotation=math.pi/2,
                                      translation=(0, 1))
print(tform.params)

####################################################################<br>
Alternatively you can define a transformation by the transformation matrix<br>
itself:

In [ ]:
matrix = tform.params.copy()
matrix[1, 2] = 2
tform2 = transform.SimilarityTransform(matrix)

####################################################################<br>
These transformation objects can then be used to apply forward and inverse<br>
coordinate transformations between the source and destination coordinate<br>
systems:

In [ ]:
coord = [1, 0]
print(tform2(coord))
print(tform2.inverse(tform(coord)))

####################################################################<br>
Image warping<br>
=============<br>
<br>
Geometric transformations can also be used to warp images:

In [ ]:
text = data.text()

In [ ]:
tform = transform.SimilarityTransform(scale=1, rotation=math.pi/4,
                                      translation=(text.shape[0]/2, -100))

In [ ]:
rotated = transform.warp(text, tform)
back_rotated = transform.warp(rotated, tform.inverse)

In [ ]:
fig, ax = plt.subplots(nrows=3)

In [ ]:
ax[0].imshow(text, cmap=plt.cm.gray)
ax[1].imshow(rotated, cmap=plt.cm.gray)
ax[2].imshow(back_rotated, cmap=plt.cm.gray)

In [ ]:
for a in ax:
    a.axis('off')

In [ ]:
plt.tight_layout()

####################################################################<br>
Parameter estimation<br>
====================<br>
<br>
In addition to the basic functionality mentioned above you can also<br>
estimate the parameters of a geometric transformation using the least-<br>
squares method.<br>
<br>
This can amongst other things be used for image registration or<br>
rectification, where you have a set of control points or<br>
homologous/corresponding points in two images.<br>
<br>
Let's assume we want to recognize letters on a photograph which was not<br>
taken from the front but at a certain angle. In the simplest case of a<br>
plane paper surface the letters are projectively distorted. Simple matching<br>
algorithms would not be able to match such symbols. One solution to this<br>
problem would be to warp the image so that the distortion is removed and<br>
then apply a matching algorithm:

In [ ]:
text = data.text()

In [ ]:
src = np.array([[0, 0], [0, 50], [300, 50], [300, 0]])
dst = np.array([[155, 15], [65, 40], [260, 130], [360, 95]])

In [ ]:
tform3 = transform.ProjectiveTransform()
tform3.estimate(src, dst)
warped = transform.warp(text, tform3, output_shape=(50, 300))

In [ ]:
fig, ax = plt.subplots(nrows=2, figsize=(8, 3))

In [ ]:
ax[0].imshow(text, cmap=plt.cm.gray)
ax[0].plot(dst[:, 0], dst[:, 1], '.r')
ax[1].imshow(warped, cmap=plt.cm.gray)

In [ ]:
for a in ax:
    a.axis('off')

In [ ]:
plt.tight_layout()
plt.show()

####################################################################<br>
The above estimation relies on accurate selection of corresponding points.<br>
An alternative approach called the<br>
`RANSAC algorithm <https://en.wikipedia.org/wiki/Random_sample_consensus>`_<br>
is useful when the correspondence points are not perfectly accurate.<br>
See the :ref:`sphx_glr_auto_examples_transform_plot_matching.py` tutorial<br>
for an in-depth description of how to use this approach in scikit-image.